In [ ]:
#Installing needed libraries
!pip install geopandas

     |████████████████████████████████| 962kB 2.8MB/s 
     |████████████████████████████████| 14.7MB 301kB/s 
     |████████████████████████████████| 10.9MB 40.9MB/s 


In [ ]:
#Importing needed libraries
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
#Loading legally opened businesses around new york
business_data = pd.read_csv('Legally_Operating_Businesses.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Visualising the data
business_data.head()

,DCA License Number,License Type,License Expiration Date,License Status,License Creation Date,Industry,Business Name,Business Name 2,Address Building,Address Street Name,Secondary Address Street Name,Address City,Address State,Address ZIP,Contact Phone Number,Address Borough,Borough Code,Community Board,Council District,BIN,BBL,NTA,Census Tract,Detail,Longitude,Latitude,Location
0,1232665-DCA,Individual,02/28/2021,Active,07/10/2006,Home Improvement Salesperson,"CATALFUMO, DANIEL J",NaN,NaN,NaN,NaN,LITTLE EGG HARBOR TWP,NJ,08087,6465339803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1217192-DCA,Individual,02/28/2021,Active,01/09/2006,Home Improvement Salesperson,"MICHILLI, ANGELO",NaN,NaN,NaN,NaN,HARRISON,NY,10528,6466139300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0868067-DCA,Individual,02/28/2021,Active,10/04/1994,Home Improvement Salesperson,"BURKE, EDWARD",NaN,NaN,NaN,NaN,DOUGLASTON,NY,11363,7182248215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012563-DCA,Individual,02/28/2021,Active,08/27/2014,Home Improvement Salesperson,"CHEN, YI FA",NaN,NaN,NaN,NaN,STATEN ISLAND,NY,10306,6464699228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2032192-DCA,Individual,02/28/2021,Active,01/06/2016,Home Improvement Salesperson,"ALEXANDER, JAROSLAV",NaN,NaN,NaN,NaN,MAPLEWOOD,NJ,07040,9733272477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Subsetting data
business_data = business_data[['DCA License Number','License Status','License Creation Date','Industry',
                               'Business Name','BBL','Census Tract','Longitude','Latitude']]

In [ ]:
#Checking for null values
business_data.isnull().sum()

DCA License Number           0
License Status               0
License Creation Date        0
Industry                     0
Business Name               38
BBL                      89978
Census Tract             93400
Longitude                81690
Latitude                 81690
dtype: int64

In [ ]:
#Dropping null values
business_data = business_data.dropna()
business_data.shape

(107377, 9)

In [ ]:
#Visualising Industry Types
business_data['Industry'].value_counts()

Tobacco Retail Dealer             23215
Home Improvement Contractor       22052
Secondhand Dealer - General       11786
Electronics Store                  9907
Stoop Line Stand                   4766
Laundries                          4325
Electronic & Appliance Service     4195
Laundry                            4178
Sidewalk Cafe                      2967
Laundry Jobber                     2939
Electronic Cigarette Dealer        2890
Garage                             2331
Dealer In Products                 1645
Secondhand Dealer - Auto           1626
Tow Truck Company                  1049
Parking Lot                         941
Amusement Device Portable           835
Employment Agency                   735
Pawnbroker                          716
Amusement Device Temporary          531
Special Sale                        480
Debt Collection Agency              430
Pedicab Business                    420
Cabaret                             318
Amusement Device Permanent          275


In [ ]:
#Creating an year column in business_data for time series analysis
business_data['year'] = pd.DatetimeIndex(business_data['License Creation Date']).year

In [ ]:
#Spatial Matching 
#Installing necessary geospatial libraries
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
!pip install pyspark
!pip install fiona
!pip install shapely

In [ ]:
#Importing needed libraries
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

import seaborn as sns
sns.set(style="whitegrid")

import fiona
import fiona.crs
import shapely
import rtree

import pandas as pd
import geopandas as gpd

In [ ]:
NYC_COUNTIES = [
    "005", # Bronx
    "047", # Kings (Brooklyn)
    "061", # New York County (Manhattan)
    "081", # Queens
    "085", # Richmond (Staten Island)
]

In [ ]:
spatial_data = gpd.read_file('tl_2019_36_tract.shp')
spatial_data['GEOID'] = spatial_data['GEOID'].astype(int)
spatial_data= spatial_data[spatial_data['COUNTYFP'].isin(NYC_COUNTIES)]
spatial_data.to_file("census_tract_ny.json", driver="GeoJSON")

In [ ]:
census_tracts = gpd.read_file('census_tract_ny.json').to_crs(fiona.crs.from_epsg(2263))

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
#Creating rtree indexes for spatial matching of BBLs and Census Tracts
index = rtree.Rtree()
for idx,geometry in enumerate(census_tracts.geometry):
    index.insert(idx, geometry.bounds)

In [ ]:
index.bounds,idx

([912190.8893736254,
  113288.79017251907,
  1067382.526638073,
  273587.03900878486],
 2166)

In [ ]:
#Spatial matching of BBLs and Census Tracts

import csv
import pyproj
import shapely.geometry as geom


proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    

counts = {}

for i, row in business_data.iterrows():
  p = geom.Point(proj(float(row['Longitude']), float(row['Latitude'])))
  match = None
  for idx in index.intersection((p.x, p.y, p.x, p.y)):
    if census_tracts.geometry[idx].contains(p):
      license = row['DCA License Number']
      match = census_tracts.GEOID[idx]
      counts[license] = match

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [ ]:
#Creating new column with Census Tract GEOIDs
geo_license = pd.DataFrame(counts.items())
geo_license = geo_license.rename(columns={0:'DCA License Number',1:'GEOID'})
geo_license.head()

,DCA License Number,GEOID
0,1385967-DCA,36061005800
1,2046821-DCA,36081054200
2,1101482-DCA,36081025500
3,2050840-DCA,36061013400
4,1465832-DCA,36061031704


In [ ]:
business_data = business_data.merge(geo_license,on='DCA License Number',how='left')

In [ ]:
business_data.isnull().sum()

DCA License Number        0
License Status            0
License Creation Date     0
Industry                  0
Business Name             0
BBL                       0
Census Tract              0
Longitude                 0
Latitude                  0
year                      0
GEOID                    16
dtype: int64

In [ ]:
business_data = business_data.dropna()
business_data['GEOID'] = business_data['GEOID'].astype(int)
business_data.head()

,DCA License Number,License Status,License Creation Date,Industry,Business Name,BBL,Census Tract,Longitude,Latitude,year,GEOID
0,1385967-DCA,Active,03/29/2011,Sidewalk Cafe,"TAPPO, INC.",1008260011,58.0,-73.990769,40.742919,2011,36061005800
1,2046821-DCA,Inactive,12/27/2016,Stoop Line Stand,MOHABIR INTERNATIONAL TRADING INC,4105460073,542.0,-73.747673,40.715521,2016,36081054200
2,1101482-DCA,Active,02/06/2002,Home Improvement Contractor,Major Air Service Corp,4011940032,255.0,-73.905945,40.752849,2002,36081025500
3,2050840-DCA,Active,04/07/2017,Sidewalk Cafe,1483 FIRST AVENUE RESTAURANT INC.,1014520024,134.0,-73.953496,40.771249,2017,36061013400
4,1465832-DCA,Active,05/29/2013,Sidewalk Cafe,"BILL'S DTM NY, LLC",1000550016,13.0,-74.014875,40.709646,2013,36061031704


In [ ]:
#Creating separate time-series datasets 
business_data_2013 = business_data[business_data['year']==2013]
business_data_2018 = business_data[business_data['year']==2018]

business_data_2018.head()

,DCA License Number,License Status,License Creation Date,Industry,Business Name,BBL,Census Tract,Longitude,Latitude,year,GEOID
6,2066708-1-DCA,Active,02/23/2018,Tobacco Retail Dealer,BEER CAVE & MART CORP,2032730257,397.0,-73.885013,40.858393,2018,36005039700
18,2066158-1-DCA,Active,02/15/2018,Tobacco Retail Dealer,BK YUMMY CORP,3037260001,908.0,-73.906528,40.669006,2018,36047090800
20,2080756-DCA,Active,12/17/2018,Secondhand Dealer - Auto,CHAMPION AUTO SALES OF THE BRONX INC.,2052510008,46201.0,-73.825625,40.882966,2018,36005046201
27,2073413-1-DCA,Inactive,06/14/2018,Electronic Cigarette Dealer,ANSHROOP CORPORATION,4027370037,603.0,-73.904270,40.712849,2018,36081060300
30,2064637-DCA,Inactive,01/08/2018,Electronic & Appliance Service,2 BROOKLYN WIRELESS INC,3048840063,824.0,-73.949515,40.650365,2018,36047082400


In [ ]:
#2018 all business data
business_all_2018 = pd.DataFrame(business_data_2018[['GEOID','Industry','Business Name']].groupby(['GEOID','Industry']).agg('count'))
business_all_2018.to_csv('business_all_18.csv')
business_all_2018.head()

Business Name
GEOID       Industry                                  
36005000200 Secondhand Dealer - General              1
            Tobacco Retail Dealer                    1
            Tow Truck Company                        1
36005000400 Electronic Cigarette Dealer              1
            Tobacco Retail Dealer                    3

In [ ]:
#2013 all business data
business_all_2013 = pd.DataFrame(business_data_2013[['GEOID','Industry','Business Name']].groupby(['GEOID','Industry']).agg('count'))
business_all_2013.to_csv('business_all_13.csv')
business_all_2013.head()

Business Name
GEOID       Industry                                  
36005000200 Tobacco Retail Dealer                    1
36005001600 Home Improvement Contractor              1
            Tobacco Retail Dealer                    1
36005001900 Home Improvement Contractor              3
            Secondhand Dealer - General              2

In [ ]:
business_all_18 = pd.read_csv('business_all_18.csv')
business_all_13 = pd.read_csv('business_all_13.csv')

In [ ]:
business_all_change = pd.merge(business_all_18,business_all_13,on=['GEOID','Industry'],how='left',suffixes=('_18','_13'))
business_all_change.head()

,GEOID,Industry,Business Name_18,Business Name_13
0,36005000200,Secondhand Dealer - General,1,NaN
1,36005000200,Tobacco Retail Dealer,1,1.0
2,36005000200,Tow Truck Company,1,NaN
3,36005000400,Electronic Cigarette Dealer,1,NaN
4,36005000400,Tobacco Retail Dealer,3,NaN


In [ ]:
business_all_change = business_all_change.dropna()
business_all_change.shape
business_all_change.head()

,GEOID,Industry,Business Name_18,Business Name_13
1,36005000200,Tobacco Retail Dealer,1,1.0
11,36005001900,Secondhand Dealer - General,1,2.0
12,36005001900,Storage Warehouse,1,1.0
14,36005002000,Tobacco Retail Dealer,1,3.0
19,36005002500,Electronics Store,2,2.0


In [ ]:
#Calculating the displacement based on inactive licenses in 2018 compared to 2013
business_all_change['pct_change'] = ((business_all_change['Business Name_18'] - business_all_change['Business Name_13'])/business_all_change['Business Name_13'])*100
business_all_change.head()

,GEOID,Industry,Business Name_18,Business Name_13,pct_change
1,36005000200,Tobacco Retail Dealer,1,1.0,0.000000
11,36005001900,Secondhand Dealer - General,1,2.0,-50.000000
12,36005001900,Storage Warehouse,1,1.0,0.000000
14,36005002000,Tobacco Retail Dealer,1,3.0,-66.666667
19,36005002500,Electronics Store,2,2.0,0.000000


In [ ]:
#Grouping the data for each GEOID
business_all_pct_change = business_all_change[['GEOID','pct_change']].groupby(['GEOID']).agg('mean')
business_all_pct_change

,pct_change
GEOID,
36005000200,0.000000
36005001900,-25.000000
36005002000,-66.666667
36005002500,-10.000000
36005002701,100.000000
...,...
36085029102,25.000000
36085029103,-33.333333
36085029104,0.000000


In [ ]:
#Getting the gentrification data for correlations
gentrification = pd.read_csv('gentrification_main.csv', usecols = ['geo_id','educational_attainment_change','home_pct_change','pct_white_alone_change','pct_black_alone_change','pct_asian_alone_change','median_income_18','median_income_13'])
gentrification = gentrification.rename(columns={'geo_id':'GEOID'})
gentrification.head()

,GEOID,median_income_18,median_income_13,educational_attainment_change,home_pct_change,pct_white_alone_change,pct_black_alone_change,pct_asian_alone_change
0,36005050400,NaN,NaN,0.000000,NaN,-42.424242,-9.950860,1.351351
1,36081005000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36005043500,NaN,NaN,2.770083,NaN,4.155125,-57.617729,0.000000
3,36081056100,100000.0,112000.0,-6.712329,NaN,43.949772,-27.397260,14.634703
4,36081011400,49242.0,44464.0,-14.624938,-0.272557,-2.780122,7.535241,20.937126


In [ ]:
#Calculating income changes
gentrification['pct_median_income_change'] = ((gentrification['median_income_18'] - gentrification['median_income_13'])/gentrification['median_income_13'])*100

In [ ]:
gentrification_change = gentrification.merge(business_all_pct_change, on = 'GEOID', how ='left')

In [ ]:
gentrification_change.head()

,GEOID,median_income_18,median_income_13,educational_attainment_change,home_pct_change,pct_white_alone_change,pct_black_alone_change,pct_asian_alone_change,pct_median_income_change,pct_change
0,36005050400,NaN,NaN,0.000000,NaN,-42.424242,-9.950860,1.351351,NaN,NaN
1,36081005000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36005043500,NaN,NaN,2.770083,NaN,4.155125,-57.617729,0.000000,NaN,NaN
3,36081056100,100000.0,112000.0,-6.712329,NaN,43.949772,-27.397260,14.634703,-10.714286,NaN
4,36081011400,49242.0,44464.0,-14.624938,-0.272557,-2.780122,7.535241,20.937126,10.745772,NaN


In [ ]:
#Getting correlation estimates
gentrification_change.corr()

,GEOID,median_income_18,median_income_13,educational_attainment_change,home_pct_change,pct_white_alone_change,pct_black_alone_change,pct_asian_alone_change,pct_median_income_change,pct_change
GEOID,1.000000,0.303850,0.313872,-0.023220,0.012251,-0.018703,0.034626,0.105824,0.004034,-0.089696
median_income_18,0.303850,1.000000,0.896636,0.021408,0.206895,-0.104144,0.071849,0.066418,0.222628,-0.036983
median_income_13,0.313872,0.896636,1.000000,-0.043966,0.105055,-0.111307,0.062306,0.073378,-0.174223,-0.036469
educational_attainment_change,-0.023220,0.021408,-0.043966,1.000000,0.081316,0.193249,-0.041139,0.044631,0.217631,0.013931
home_pct_change,0.012251,0.206895,0.105055,0.081316,1.000000,0.024417,0.009142,0.021303,0.243298,-0.016289
pct_white_alone_change,-0.018703,-0.104144,-0.111307,0.193249,0.024417,1.000000,-0.252279,-0.349712,0.085208,0.000972
pct_black_alone_change,0.034626,0.071849,0.062306,-0.041139,0.009142,-0.252279,1.000000,-0.080950,0.019574,0.023635
pct_asian_alone_change,0.105824,0.066418,0.073378,0.044631,0.021303,-0.349712,-0.080950,1.000000,-0.012214,-0.009760
pct_median_income_change,0.004034,0.222628,-0.174223,0.217631,0.243298,0.085208,0.019574,-0.012214,1.000000,0.003785
pct_change,-0.089696,-0.036983,-0.036469,0.013931,-0.016289,0.000972,0.023635,-0.009760,0.003785,1.000000


### Getting correlation estimates for Tobacco Retailers

In [ ]:
#Creating separate time-series datasets 
tobacco_data_2013 = business_data[(business_data['year']==2013)&(business_data['Industry']=='Tobacco Retail Dealer')]
tobacco_data_2018 = business_data[(business_data['year']==2018)&(business_data['Industry']=='Tobacco Retail Dealer')]

tobacco_data_2018.head()

,DCA License Number,License Status,License Creation Date,Industry,Business Name,BBL,Census Tract,Longitude,Latitude,year,GEOID
6,2066708-1-DCA,Active,02/23/2018,Tobacco Retail Dealer,BEER CAVE & MART CORP,2032730257,397.0,-73.885013,40.858393,2018,36005039700
18,2066158-1-DCA,Active,02/15/2018,Tobacco Retail Dealer,BK YUMMY CORP,3037260001,908.0,-73.906528,40.669006,2018,36047090800
77,2066655-1-DCA,Active,02/23/2018,Tobacco Retail Dealer,ROHAN DELI INC.,4157500001,100802.0,-73.753706,40.600948,2018,36081100802
181,2066860-1-DCA,Active,02/23/2018,Tobacco Retail Dealer,LOS MOROCHOS GROCERY STORE INC,2028540024,241.0,-73.908119,40.851656,2018,36005024100
199,2065909-1-DCA,Inactive,02/09/2018,Tobacco Retail Dealer,SNAGGER INC,3073430075,582.0,-73.961135,40.597769,2018,36047058200


In [ ]:
#2018 tobacco business data
tobacco_2018 = pd.DataFrame(tobacco_data_2018[['GEOID','Business Name']].groupby(['GEOID']).agg('count'))
tobacco_2018.to_csv('tobacco_2018.csv')
tobacco_2018.head()

,Business Name
GEOID,
36005000200,1
36005000400,3
36005002000,1
36005002500,4
36005002701,2


In [ ]:
#2013 tobacco business data
tobacco_2013 = pd.DataFrame(tobacco_data_2013[['GEOID','Business Name']].groupby(['GEOID']).agg('count'))
tobacco_2013.to_csv('tobacco_2013.csv')
tobacco_2013.head()

,Business Name
GEOID,
36005000200,1
36005001600,1
36005002000,3
36005002500,5
36005002701,1


In [ ]:
tobacco_2018 = pd.read_csv('tobacco_2018.csv')
tobacco_2013 = pd.read_csv('tobacco_2013.csv')
tobacco_change = pd.merge(tobacco_2018,tobacco_2013,on=['GEOID'],how='left',suffixes=('_18','_13'))
tobacco_change.head()

,GEOID,Business Name_18,Business Name_13
0,36005000200,1,1.0
1,36005000400,3,NaN
2,36005002000,1,3.0
3,36005002500,4,5.0
4,36005002701,2,1.0


In [ ]:
tobacco_change = tobacco_change.dropna()
tobacco_change['pct_change'] = ((tobacco_change['Business Name_18'] - tobacco_change['Business Name_13'])/tobacco_change['Business Name_13'])*100
tobacco_change.head()

,GEOID,Business Name_18,Business Name_13,pct_change
0,36005000200,1,1.0,0.000000
2,36005002000,1,3.0,-66.666667
3,36005002500,4,5.0,-20.000000
4,36005002701,2,1.0,100.000000
5,36005003900,4,2.0,100.000000


In [ ]:
gentrification_change = gentrification.merge(tobacco_change, on = 'GEOID', how ='left')
gentrification_change.head()

,GEOID,median_income_18,median_income_13,educational_attainment_change,home_pct_change,pct_white_alone_change,pct_black_alone_change,pct_asian_alone_change,pct_median_income_change,Business Name_18,Business Name_13,pct_change
0,36005050400,NaN,NaN,0.000000,NaN,-42.424242,-9.950860,1.351351,NaN,NaN,NaN,NaN
1,36081005000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36005043500,NaN,NaN,2.770083,NaN,4.155125,-57.617729,0.000000,NaN,NaN,NaN,NaN
3,36081056100,100000.0,112000.0,-6.712329,NaN,43.949772,-27.397260,14.634703,-10.714286,NaN,NaN,NaN
4,36081011400,49242.0,44464.0,-14.624938,-0.272557,-2.780122,7.535241,20.937126,10.745772,NaN,NaN,NaN


In [ ]:
gentrification_change.corr()

,GEOID,median_income_18,median_income_13,educational_attainment_change,home_pct_change,pct_white_alone_change,pct_black_alone_change,pct_asian_alone_change,pct_median_income_change,Business Name_18,Business Name_13,pct_change
GEOID,1.000000,0.303850,0.313872,-0.023220,0.012251,-0.018703,0.034626,0.105824,0.004034,-0.179415,-0.061531,-0.106516
median_income_18,0.303850,1.000000,0.896636,0.021408,0.206895,-0.104144,0.071849,0.066418,0.222628,-0.157013,-0.083715,-0.078377
median_income_13,0.313872,0.896636,1.000000,-0.043966,0.105055,-0.111307,0.062306,0.073378,-0.174223,-0.181003,-0.113815,-0.079346
educational_attainment_change,-0.023220,0.021408,-0.043966,1.000000,0.081316,0.193249,-0.041139,0.044631,0.217631,-0.023804,0.063205,-0.035947
home_pct_change,0.012251,0.206895,0.105055,0.081316,1.000000,0.024417,0.009142,0.021303,0.243298,-0.020768,0.013341,-0.015667
pct_white_alone_change,-0.018703,-0.104144,-0.111307,0.193249,0.024417,1.000000,-0.252279,-0.349712,0.085208,-0.010027,0.105422,-0.070719
pct_black_alone_change,0.034626,0.071849,0.062306,-0.041139,0.009142,-0.252279,1.000000,-0.080950,0.019574,0.083437,0.021822,0.074950
pct_asian_alone_change,0.105824,0.066418,0.073378,0.044631,0.021303,-0.349712,-0.080950,1.000000,-0.012214,-0.012715,0.033938,-0.044057
pct_median_income_change,0.004034,0.222628,-0.174223,0.217631,0.243298,0.085208,0.019574,-0.012214,1.000000,0.038230,0.053567,0.006694
Business Name_18,-0.179415,-0.157013,-0.181003,-0.023804,-0.020768,-0.010027,0.083437,-0.012715,0.038230,1.000000,0.182576,0.637346


### Getting correlation estimates for Cafes

In [ ]:
#Creating separate time-series datasets 
cafe_data_2013 = business_data[(business_data['year']==2013)&(business_data['Industry']=='Sidewalk Cafe') | (business_data['Industry']=='Gaming Cafe')]
cafe_data_2018 = business_data[(business_data['year']==2018)&(business_data['Industry']=='Sidewalk Cafe') | (business_data['Industry']=='Gaming Cafe')]

cafe_data_2018.head()

,DCA License Number,License Status,License Creation Date,Industry,Business Name,BBL,Census Tract,Longitude,Latitude,year,GEOID
60,2080703-DCA,Active,12/14/2018,Sidewalk Cafe,LLAMA SAN LLC,1005920019,71.0,-74.000649,40.732143,2018,36061007100
183,2071755-DCA,Inactive,05/23/2018,Sidewalk Cafe,STRUZZO LLC,3017870036,281.0,-73.940313,40.691884,2018,36047028100
192,2078751-DCA,Active,10/02/2018,Sidewalk Cafe,BURGEROLOGY ASTORIA LLC,4008310099,113.0,-73.910148,40.775760,2018,36081011300
622,2077830-DCA,Active,09/04/2018,Sidewalk Cafe,834 7TH AVENUE LLC,1010257501,131.0,-73.981633,40.763338,2018,36061013100
2045,2068210-DCA,Inactive,03/22/2018,Sidewalk Cafe,OIDIA LLC,1006270012,79.0,-74.005453,40.739084,2018,36061007900


In [ ]:
#2018 cafe business data
cafe_2018 = pd.DataFrame(cafe_data_2018[['GEOID','Business Name']].groupby(['GEOID']).agg('count'))
cafe_2018.to_csv('cafe_2018.csv')
cafe_2018.head()

,Business Name
GEOID,
36005016200,1
36005023702,1
36005028300,1
36005038600,1
36005039500,1


In [ ]:
#2013 cafe business data
cafe_2013 = pd.DataFrame(cafe_data_2013[['GEOID','Business Name']].groupby(['GEOID']).agg('count'))
cafe_2013.to_csv('cafe_2013.csv')
cafe_2013.head()

,Business Name
GEOID,
36005013200,1
36005019400,1
36005023702,1
36005028300,2
36005029500,1


In [ ]:
cafe_2018 = pd.read_csv('cafe_2018.csv')
cafe_2013 = pd.read_csv('cafe_2013.csv')
cafe_change = pd.merge(cafe_2018,cafe_2013,on=['GEOID'],how='left',suffixes=('_18','_13'))
cafe_change.head()

,GEOID,Business Name_18,Business Name_13
0,36005016200,1,NaN
1,36005023702,1,1.0
2,36005028300,1,2.0
3,36005038600,1,1.0
4,36005039500,1,1.0


In [ ]:
cafe_change = cafe_change.dropna()
cafe_change['pct_change'] = ((cafe_change['Business Name_18'] - cafe_change['Business Name_13'])/cafe_change['Business Name_13'])*100
cafe_change.head()

,GEOID,Business Name_18,Business Name_13,pct_change
1,36005023702,1,1.0,0.0
2,36005028300,1,2.0,-50.0
3,36005038600,1,1.0,0.0
4,36005039500,1,1.0,0.0
8,36047007800,1,1.0,0.0


In [ ]:
gentrification_change = gentrification.merge(cafe_change, on = 'GEOID', how ='left')
gentrification_change.head()

,GEOID,median_income_18,median_income_13,educational_attainment_change,home_pct_change,pct_white_alone_change,pct_black_alone_change,pct_asian_alone_change,pct_median_income_change,Business Name_18,Business Name_13,pct_change
0,36005050400,NaN,NaN,0.000000,NaN,-42.424242,-9.950860,1.351351,NaN,NaN,NaN,NaN
1,36081005000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36005043500,NaN,NaN,2.770083,NaN,4.155125,-57.617729,0.000000,NaN,NaN,NaN,NaN
3,36081056100,100000.0,112000.0,-6.712329,NaN,43.949772,-27.397260,14.634703,-10.714286,NaN,NaN,NaN
4,36081011400,49242.0,44464.0,-14.624938,-0.272557,-2.780122,7.535241,20.937126,10.745772,NaN,NaN,NaN


In [ ]:
gentrification_change.corr()

,GEOID,median_income_18,median_income_13,educational_attainment_change,home_pct_change,pct_white_alone_change,pct_black_alone_change,pct_asian_alone_change,pct_median_income_change,Business Name_18,Business Name_13,pct_change
GEOID,1.000000,0.303850,0.313872,-0.023220,0.012251,-0.018703,0.034626,0.105824,0.004034,0.041891,0.027726,0.040745
median_income_18,0.303850,1.000000,0.896636,0.021408,0.206895,-0.104144,0.071849,0.066418,0.222628,-0.082981,-0.045959,0.006596
median_income_13,0.313872,0.896636,1.000000,-0.043966,0.105055,-0.111307,0.062306,0.073378,-0.174223,-0.130145,-0.112036,0.014848
educational_attainment_change,-0.023220,0.021408,-0.043966,1.000000,0.081316,0.193249,-0.041139,0.044631,0.217631,0.013228,0.048803,-0.019528
home_pct_change,0.012251,0.206895,0.105055,0.081316,1.000000,0.024417,0.009142,0.021303,0.243298,-0.003891,0.009625,-0.031136
pct_white_alone_change,-0.018703,-0.104144,-0.111307,0.193249,0.024417,1.000000,-0.252279,-0.349712,0.085208,0.149117,0.236702,-0.228671
pct_black_alone_change,0.034626,0.071849,0.062306,-0.041139,0.009142,-0.252279,1.000000,-0.080950,0.019574,-0.016884,-0.009474,-0.021432
pct_asian_alone_change,0.105824,0.066418,0.073378,0.044631,0.021303,-0.349712,-0.080950,1.000000,-0.012214,-0.149267,-0.192542,0.042940
pct_median_income_change,0.004034,0.222628,-0.174223,0.217631,0.243298,0.085208,0.019574,-0.012214,1.000000,0.159507,0.182705,0.014190
Business Name_18,0.041891,-0.082981,-0.130145,0.013228,-0.003891,0.149117,-0.016884,-0.149267,0.159507,1.000000,0.938050,0.198592


### Getting correlation estimates for Home Improvement Businesses

In [ ]:
#Creating separate time-series datasets 
home_data_2013 = business_data[(business_data['year']==2013)&(business_data['Industry']=='Home Improvement Contractor')]
home_data_2018 = business_data[(business_data['year']==2018)&(business_data['Industry']=='Home Improvement Contractor')]

home_data_2018.head()

,DCA License Number,License Status,License Creation Date,Industry,Business Name,BBL,Census Tract,Longitude,Latitude,year,GEOID
620,2080662-DCA,Inactive,12/13/2018,Home Improvement Contractor,"TAPIA, RAMIRO",4020130062,399.0,-73.854960,40.751056,2018,36081039900
2512,2080097-DCA,Active,11/21/2018,Home Improvement Contractor,Breuckelen Construction Group,3004710125,77.0,-73.992966,40.676624,2018,36047007700
3276,2077815-DCA,Active,09/04/2018,Home Improvement Contractor,BIOPHILIC BUILD LLC,3015660007,301.0,-73.912704,40.676957,2018,36047030100
8547,2067187-DCA,Active,03/02/2018,Home Improvement Contractor,MASTER INSTALL INC,2028450007,209.0,-73.914915,40.842477,2018,36005020900
9317,2079541-DCA,Active,10/29/2018,Home Improvement Contractor,GLR Home Improvement Inc.,4044940036,981.0,-73.819006,40.797128,2018,36081098100


In [ ]:
#2018 cafe business data
home_2018 = pd.DataFrame(home_data_2018[['GEOID','Business Name']].groupby(['GEOID']).agg('count'))
home_2018.to_csv('home_2018.csv')
#2013 cafe business data
home_2013 = pd.DataFrame(home_data_2013[['GEOID','Business Name']].groupby(['GEOID']).agg('count'))
home_2013.to_csv('home_2013.csv')
home_2013.head()

,Business Name
GEOID,
36005001600,1
36005001900,3
36005003100,4
36005004800,1
36005005001,2


In [ ]:
home_2018 = pd.read_csv('home_2018.csv')
home_2013 = pd.read_csv('home_2013.csv')
home_change = pd.merge(home_2018,home_2013,on=['GEOID'],how='left',suffixes=('_18','_13'))
home_change.head()

,GEOID,Business Name_18,Business Name_13
0,36005003100,1,4.0
1,36005003500,1,NaN
2,36005004300,1,NaN
3,36005004800,2,1.0
4,36005005002,1,NaN


In [ ]:
home_change = home_change.dropna()
home_change['pct_change'] = ((home_change['Business Name_18'] - home_change['Business Name_13'])/home_change['Business Name_13'])*100
home_change.head()

,GEOID,Business Name_18,Business Name_13,pct_change
0,36005003100,1,4.0,-75.0
3,36005004800,2,1.0,100.0
5,36005005100,1,1.0,0.0
7,36005005400,2,1.0,100.0
9,36005006700,1,1.0,0.0


In [ ]:
gentrification_change = gentrification.merge(home_change, on = 'GEOID', how ='left')
gentrification_change.head()

,GEOID,median_income_18,median_income_13,educational_attainment_change,home_pct_change,pct_white_alone_change,pct_black_alone_change,pct_asian_alone_change,pct_median_income_change,Business Name_18,Business Name_13,pct_change
0,36005050400,NaN,NaN,0.000000,NaN,-42.424242,-9.950860,1.351351,NaN,NaN,NaN,NaN
1,36081005000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36005043500,NaN,NaN,2.770083,NaN,4.155125,-57.617729,0.000000,NaN,NaN,NaN,NaN
3,36081056100,100000.0,112000.0,-6.712329,NaN,43.949772,-27.397260,14.634703,-10.714286,NaN,NaN,NaN
4,36081011400,49242.0,44464.0,-14.624938,-0.272557,-2.780122,7.535241,20.937126,10.745772,NaN,NaN,NaN


In [ ]:
gentrification_change.corr()

,GEOID,median_income_18,median_income_13,educational_attainment_change,home_pct_change,pct_white_alone_change,pct_black_alone_change,pct_asian_alone_change,pct_median_income_change,Business Name_18,Business Name_13,pct_change
GEOID,1.000000,0.303850,0.313872,-0.023220,0.012251,-0.018703,0.034626,0.105824,0.004034,0.048005,0.172829,-0.112725
median_income_18,0.303850,1.000000,0.896636,0.021408,0.206895,-0.104144,0.071849,0.066418,0.222628,0.009141,0.076746,-0.047034
median_income_13,0.313872,0.896636,1.000000,-0.043966,0.105055,-0.111307,0.062306,0.073378,-0.174223,-0.003337,0.093000,-0.079020
educational_attainment_change,-0.023220,0.021408,-0.043966,1.000000,0.081316,0.193249,-0.041139,0.044631,0.217631,0.012482,0.004099,0.022792
home_pct_change,0.012251,0.206895,0.105055,0.081316,1.000000,0.024417,0.009142,0.021303,0.243298,0.014981,0.062960,-0.024460
pct_white_alone_change,-0.018703,-0.104144,-0.111307,0.193249,0.024417,1.000000,-0.252279,-0.349712,0.085208,-0.050230,-0.001533,-0.023163
pct_black_alone_change,0.034626,0.071849,0.062306,-0.041139,0.009142,-0.252279,1.000000,-0.080950,0.019574,0.047276,-0.025570,0.046133
pct_asian_alone_change,0.105824,0.066418,0.073378,0.044631,0.021303,-0.349712,-0.080950,1.000000,-0.012214,0.031352,0.001654,-0.020815
pct_median_income_change,0.004034,0.222628,-0.174223,0.217631,0.243298,0.085208,0.019574,-0.012214,1.000000,0.011417,-0.023103,0.044427
Business Name_18,0.048005,0.009141,-0.003337,0.012482,0.014981,-0.050230,0.047276,0.031352,0.011417,1.000000,0.273741,0.575791
